In [451]:
"""
MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 20 YEARS OF DATA

First,  we use the backtesting library to find the best crossover strategy using 20 years of data.
Next, we confirm the results using our own backtesting and metrics. 

The backtesting library choose short = 215, long = 225. It tested 2556 combinations using short = 5 to 365 with a step of 5
and long = 5 to 365 with a step of 5. 

"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

%matplotlib inline

In [452]:
#load alpaca keys

load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [453]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-07-02,314.24,315.70,312.0400,314.1998,314.1998,42771431.0,0.0,1.0
2020-07-01,309.57,311.85,309.0371,310.5700,310.5700,66883431.0,0.0,1.0
2020-06-30,303.99,310.20,303.8200,308.3600,308.3600,112828251.0,0.0,1.0
2020-06-29,301.41,304.61,298.9300,304.4600,304.4600,79411577.0,0.0,1.0
2020-06-26,306.16,306.39,299.4200,300.0500,300.0500,127811745.0,0.0,1.0


In [454]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2000-06-30,143.9375,145.5937,143.8906,145.2812,99.2910,7420200.0,0.0,1.0
2000-07-03,145.4375,147.4375,145.1875,147.2812,100.6579,1436600.0,0.0,1.0
2000-07-05,146.3750,146.6562,144.3750,144.6250,98.8425,2748200.0,0.0,1.0
2000-07-06,144.9375,146.4687,144.2187,145.7500,99.6114,5963200.0,0.0,1.0
2000-07-07,146.6875,148.7812,146.2500,148.0937,101.2132,3034800.0,0.0,1.0


In [455]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.sort_index(ascending=True, inplace=True)
stock_data_df.head()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
2000-06-30,143.9375,145.5937,143.8906,99.2910,7420200.0
2000-07-03,145.4375,147.4375,145.1875,100.6579,1436600.0
2000-07-05,146.3750,146.6562,144.3750,98.8425,2748200.0
2000-07-06,144.9375,146.4687,144.2187,99.6114,5963200.0
2000-07-07,146.6875,148.7812,146.2500,101.2132,3034800.0


In [330]:
#Function to do a backtest using Backtesting library. This function will find the best SMA cross strategy by analyzing
#different combinations of long short smas.

#Change the ranges in the stats for different scenarios. For example, range(5,365,5) means to test smas 
#between 5 and 365 days and use a 5 day step. 

def backtest():
    class SmaCross(Strategy):
    
    #n1 =short, n2 = long, they can be anything at the start of the optimization
        n1=1
        n2=365
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=100000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(5, 365, 5),
                    n2=range(5, 365, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:28: UserWarning:

Searching best of 2556 configurations.



Start                         2000-06-30 00:00:00
End                           2020-07-01 00:00:00
Duration                       7306 days 00:00:00
Exposure [%]                              87.7772
Equity Final [$]                           568889
Equity Peak [$]                            613258
Return [%]                                468.889
Buy & Hold Return [%]                     212.788
Max. Drawdown [%]                        -52.4712
Avg. Drawdown [%]                        -4.34016
Max. Drawdown Duration         1348 days 00:00:00
Avg. Drawdown Duration           58 days 00:00:00
# Trades                                       30
Win Rate [%]                              66.6667
Best Trade [%]                            66.1451
Worst Trade [%]                          -12.5163
Avg. Trade [%]                             7.2533
Max. Trade Duration            1294 days 00:00:00
Avg. Trade Duration             214 days 00:00:00
Expectancy [%]                            9.09299


In [456]:
# To confirm the above results (short = 215, long = 225) we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2000-06-30,99.2910
2000-07-03,100.6579
2000-07-05,98.8425
2000-07-06,99.6114
2000-07-07,101.2132


In [458]:
#Generate signals and put in dataframe

# Set the short window and long windows
short_window = 215
long_window = 225

#Set names of the windows
short_name = "SMA215"
long_name = "SMA225"

# Generate the short and long moving averages 
signals_df[short_name] = signals_df["Close"].rolling(window=short_window).mean()
signals_df[long_name] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df[short_name][short_window:] > signals_df[long_name][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy/Sell"] = signals_df["Signal"].diff()

signals_df.head(10)


,Close,SMA215,SMA225,Signal,Buy/Sell
date,,,,,
2000-06-30,99.2910,NaN,NaN,0.0,NaN
2000-07-03,100.6579,NaN,NaN,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0
2000-07-10,101.0423,NaN,NaN,0.0,0.0
2000-07-11,101.2559,NaN,NaN,0.0,0.0
2000-07-12,101.9180,NaN,NaN,0.0,0.0
2000-07-13,102.3665,NaN,NaN,0.0,0.0


In [459]:
len(signals_df)

5033

In [460]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA215', 'SMA225']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [462]:
signals_df.head()

,Close,SMA215,SMA225,Signal,Buy/Sell
date,,,,,
2000-06-30,99.2910,NaN,NaN,0.0,NaN
2000-07-03,100.6579,NaN,NaN,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0


In [463]:

# Add share and cash columns

signals_df["Shares"] = float(0)
signals_df["Portfolio Cash"] = float(100000)

#Set up starting cash variable for sma strategy and buy and hold strategy

cash = 100000
buy_and_hold_cash = 100000


#Iterate through each row, use combinations of signal and buy/sell points to set shares and portfolio cash for each row.

for index, row in signals_df.iterrows():
    
    #If at a buy point, calculate the number of shares using total cash available and set the cash to zero
    
    if row['Buy/Sell'] == 1.0:
       
        shares = cash/row['Close']
        row['Shares'] = shares
        row['Portfolio Cash']= 0 
       
    #If at sell point, calculate the cash generated from the sale (shares X closing price) and set the shares to zero, then
    #update the cash variable with the new cash balance'''
    
    elif row['Buy/Sell'] == -1.0:
        row['Portfolio Cash']= shares*row['Close']
        cash = row['Portfolio Cash']
        row['Shares']= 0
        
    #If we are not at a buy or sell point, look to signal column, if the signal is buy, then set cash to zero and update shares.
    # If the signal is sell, update the cash column using cash variable
    else:
     
        row['Buy/Sell'] == 0
        if row['Signal']== 1:
            row['Portfolio Cash'] = 0
            row["Shares"]= shares
        elif row['Signal'] == 0:
            row['Portfolio Cash'] = cash
        
# Add a Portfolio Total column
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + (signals_df['Shares']*signals_df['Close'])

# Calculate how many shares a buy and hold investor could buy at the start of the comparison. Use end of long window as start. 
buy_and_hold_shares = buy_and_hold_cash/signals_df['Close'].iloc[long_window-1]

# Add a buy and hold total column for comparison, set the first 225 (long window) to 100000 so both portfolios are at 100000 at the
#end of the long window calculation
signals_df["Buy&Hold Total"]= buy_and_hold_shares * signals_df['Close']
signals_df["Buy&Hold Total"][0:long_window-1] = buy_and_hold_cash      
        
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()  
signals_df['Buy&Hold Daily Returns']= signals_df['Buy&Hold Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cum Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
signals_df['Buy&Hold Cum Returns'] = (1 + signals_df['Buy&Hold Daily Returns']).cumprod() - 1

# Print the DataFrame
pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(50)

,Close,SMA215,SMA225,Signal,Buy/Sell,Shares,Portfolio Cash,Portfolio Total,Buy&Hold Total,Portfolio Daily Returns,Buy&Hold Daily Returns,Portfolio Cum Returns,Buy&Hold Cum Returns
date,,,,,,,,,,,,,
2000-06-30,99.2910,NaN,NaN,0.0,NaN,0.0,100000.0,100000.0,100000.0,NaN,NaN,NaN,NaN
2000-07-03,100.6579,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-10,101.0423,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-11,101.2559,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-12,101.9180,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0
2000-07-13,102.3665,NaN,NaN,0.0,0.0,0.0,100000.0,100000.0,100000.0,0.0,0.0,0.0,0.0


In [464]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [465]:
#Plot the SMA Strategy vs the Buy & Hold strategy

signals_df[["Buy&Hold Total", "Portfolio Total"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [466]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest', 'Buy&Hold']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,NaN,NaN
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [467]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'][0] = signals_df['Portfolio Cum Returns'][-1]
portfolio_evaluation_df.loc['Cumulative Returns'][1] = signals_df['Buy&Hold Cum Returns'][-1]

portfolio_evaluation_df



,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,4.2322,2.46667
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [468]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'][0] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Return'][1] = (signals_df['Buy&Hold Daily Returns'].mean() * 252)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0921462,0.0806558
Cumulative Returns,4.2322,2.46667
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [469]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'][0] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Annual Volatility'][1] = (signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0921462,0.0806558
Cumulative Returns,4.2322,2.46667
Annual Volatility,0.135817,0.191703
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [470]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'][0] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df.loc['Sharpe Ratio'][1] = (
    signals_df['Buy&Hold Daily Returns'].mean() * 252) / (
    signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252)
)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0921462,0.0806558
Cumulative Returns,4.2322,2.46667
Annual Volatility,0.135817,0.191703
Sharpe Ratio,0.678457,0.420734
Sortino Ratio,NaN,NaN


In [471]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

sortino_ratio_df2 = signals_df[['Buy&Hold Daily Returns']].copy()
sortino_ratio_df2.loc[:,'Downside Returns1'] = 0

target = 0
mask = sortino_ratio_df2['Buy&Hold Daily Returns'] < target
sortino_ratio_df2.loc[mask, 'Downside Returns1'] = sortino_ratio_df2['Buy&Hold Daily Returns']**2


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0921462,0.0806558
Cumulative Returns,4.2322,2.46667
Annual Volatility,0.135817,0.191703
Sharpe Ratio,0.678457,0.420734
Sortino Ratio,NaN,NaN


In [472]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][0] = sortino_ratio

down_stdev = np.sqrt(sortino_ratio_df2['Downside Returns1'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df2['Buy&Hold Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][1] = sortino_ratio





portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0921462,0.0806558
Cumulative Returns,4.2322,2.46667
Annual Volatility,0.135817,0.191703
Sharpe Ratio,0.678457,0.420734
Sortino Ratio,0.94619,0.591427


In [473]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [474]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Buy/Sell'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Shares']*row['Close']
        share_size = row['Shares']
        entry_share_price = row['Close']

    elif row['Buy/Sell'] == -1:
        exit_date = index
        exit_portfolio_holding = row['Portfolio Total']
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2002-02-04,2002-03-04,1307.006732,76.5107,80.6182,100000.000000,105368.530153,5368.530153
1,SPY,2002-07-29,2002-08-23,1675.257925,62.8969,66.2810,105368.530153,111037.770496,5669.240342
2,SPY,2003-05-30,2003-06-26,1613.137961,68.8334,70.4010,111037.770496,113566.525563,2528.755067
3,SPY,2003-07-18,2004-11-26,1601.628693,70.9069,86.2802,113566.525563,138188.843947,24622.318384
4,SPY,2004-12-15,2004-12-17,1568.105695,88.1247,87.4889,138188.843947,137191.842346,-997.001601
5,SPY,2005-01-19,2008-03-06,1584.289226,86.5952,101.3263,137191.842346,160530.165357,23338.323011
6,SPY,2009-09-28,2010-11-19,1874.045384,85.6597,98.8290,160530.165357,185210.031229,24679.865872
7,SPY,2010-11-29,2011-02-09,1891.817121,97.9006,109.2424,185210.031229,206666.642651,21456.611422
8,SPY,2011-03-14,2011-11-25,1924.110969,107.4089,97.4773,206666.642651,187557.142152,-19109.500499
9,SPY,2012-04-23,2012-05-09,1619.073017,115.8423,114.9531,187557.142152,186117.462425,-1439.679727


In [475]:
#Create plots

price_df = signals_df[['Close', 'SMA215', 'SMA225']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [476]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest,Buy&Hold]

In [477]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [478]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [449]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)